# Create Datasets for the Content-based Filter

This notebook builds the data you will use for creating our content based model. You'll collect the data via a collection of SQL queries from the publicly available Kurier.at dataset in BigQuery.
Kurier.at is an Austrian newsite. The goal of these labs is to recommend an article for a visitor to the site. In this notebook, you collect the data for training, in the subsequent notebook you train the recommender model. 

This notebook illustrates:
* How to pull data from BigQuery table and write to local files.
* How to make reproducible train and test splits.

In [1]:
import os
import tensorflow as tf
import numpy as np
from google.cloud import bigquery 

PROJECT = 'qwiklabs-gcp-03-e64d14bc31d2' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-03-e64d14bc31d2' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'

In [2]:
%%bash
gcloud  config  set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


You will use this helper function to write lists containing article ids, categories, and authors for each article in our database to local file. 

In [3]:
def write_list_to_disk(my_list, filename):
  with open(filename, 'w') as f:
    for item in my_list:
        line = "%s\n" % item
        f.write(line)

### Pull data from BigQuery

The cell below creates a local text file containing all the article ids (i.e. 'content ids') in the dataset. 

Have a look at the original dataset in [BigQuery](https://console.cloud.google.com/bigquery?p=cloud-training-demos&d=GA360_test&t=ga_sessions_sample). Then read through the query below and make sure you understand what it is doing. 

In [4]:
sql="""
#standardSQL

SELECT  
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id 
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY
  content_id
  
"""

content_ids_list = bigquery.Client().query(sql).to_dataframe()['content_id'].tolist()
write_list_to_disk(content_ids_list, "content_ids.txt")
print("Some sample content IDs {}".format(content_ids_list[:3]))
print("The total number of articles is {}".format(len(content_ids_list)))

Some sample content IDs ['299772450', '299777082', '299777722']
The total number of articles is 15634


There should be 15,634 articles in the database.  
Next, you'll create a local file which contains a list of article categories and a list of article authors.

Note the change in the index when pulling the article category or author information. Also, you are using the first author of the article to create our author list.  
Refer back to the original dataset, use the `hits.customDimensions.index` field to verify the correct index.	 

In [5]:
sql="""
#standardSQL
SELECT  
  (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  category
"""
categories_list = bigquery.Client().query(sql).to_dataframe()['category'].tolist()
write_list_to_disk(categories_list, "categories.txt")
print(categories_list)

['Stars & Kultur', 'News', 'Lifestyle']


The categories are 'News', 'Stars & Kultur', and 'Lifestyle'.  
When creating the author list, you'll only use the first author information for each article.  

In [6]:
sql="""
#standardSQL
SELECT
  REGEXP_EXTRACT((SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)), r"^[^,]+")  AS first_author  
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
  UNNEST(hits) AS hits
WHERE 
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY   
  first_author
"""
authors_list = bigquery.Client().query(sql).to_dataframe()['first_author'].tolist()
write_list_to_disk(authors_list, "authors.txt")
print("Some sample authors {}".format(authors_list[:10]))
print("The total number of authors is {}".format(len(authors_list)))

Some sample authors ['Elisabeth Spitzer', 'Maria Zelenko', 'Ute Brühl', 'Georg Leyrer', 'Mirad Odobasic', 'Peter Temel', 'Margaretha Kopeinig', 'Werner Rosenberger', 'Raffaela Lindorfer', 'Johanna Kreid']
The total number of authors is 385


There should be 385 authors in the database. 

### Create train and test sets

In this section, you will create the train/test split of our data for training our model. You use the concatenated values for visitor id and content id to create a farm fingerprint, taking approximately 90% of the data for the training set and 10% for the test set.

In [7]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(CONCAT(visitor_id, content_id)), 10)) < 9
"""
training_set_df = bigquery.Client().query(sql).to_dataframe()
training_set_df.to_csv('training_set.csv', header=False, index=False, encoding='utf-8')
training_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1000196974485173657,299925086,News,Marihuana-Adventkalender findet in Kanada reiß...,None,574,299826775
1,1000196974485173657,299950903,News,Vienna: OGH bestätigt Zwangsabstieg in 2. Land...,Stefan Berndl,574,299965875
2,1000196974485173657,299965875,News,Finanzinvestor fordert Ende der Fusionsgespräc...,Irmgard Kischko,574,299925700
3,1000196974485173657,299930032,News,Google steigt bei Wiener IT-Firma ein,Stefan Hofer,574,299826767
4,1000196974485173657,299912085,News,Erster ÖBB-Containerzug nach China unterwegs,Stefan Hofer,574,299804319


In [8]:
sql="""
WITH site_history as (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index=7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index=6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index=2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index=4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') as year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) as nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
)
SELECT
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") as title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") as author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970,1,1), MONTH) as months_since_epoch,
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) as next_content_id
FROM
  site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(CONCAT(visitor_id, content_id)), 10)) >= 9
"""
test_set_df = bigquery.Client().query(sql).to_dataframe()
test_set_df.to_csv('test_set.csv', header=False, index=False, encoding='utf-8')
test_set_df.head()

,visitor_id,content_id,category,title,author,months_since_epoch,next_content_id
0,1084677847334161817,299830996,News,Wie die Schule in der Neuzeit ankommen könnte,Martina Salomon,574,299917726
1,11165257372406196566,299825001,News,Auslieferung: Mutmaßlicher Sechsfach-Mörder ha...,Michaela Reibenwein,574,299469182
2,1186036223658977802,298145180,News,"Adventmärkte: ""Poller schrecken Terroristen ni...",Daniel Melcher,574,299902870
3,1256061499304903979,239460999,Stars & Kultur,O.J. Simpsons Justizgroteske Folge für Folge,Philipp Wilhelmer,564,299802565
4,1256912942211654061,299982579,News,VIDEO: Basejumper springen von Berg in Flugzeug,Mathias Kainz,574,299965853


Let's have a look at the two csv files you just created containing the training and test set. You'll also do a line count of both files to confirm that you have achieved an approximate 90/10 train/test split.  
In the next notebook, **Content Based Filtering** you will build a model to recommend an article given information about the current article being read, such as the category, title, author, and publish date. 

In [9]:
%%bash
wc -l *_set.csv

   25599 test_set.csv
  232308 training_set.csv
  257907 total


In [10]:
!head *_set.csv

==> test_set.csv <==
1084677847334161817,299830996,News,Wie die Schule in der Neuzeit ankommen könnte,Martina Salomon,574,299917726
11165257372406196566,299825001,News,Auslieferung: Mutmaßlicher Sechsfach-Mörder hat Flugangst,Michaela Reibenwein,574,299469182
1186036223658977802,298145180,News,"Adventmärkte: ""Poller schrecken Terroristen nicht ab""",Daniel Melcher,574,299902870
1256061499304903979,239460999,Stars & Kultur,O.J. Simpsons Justizgroteske Folge für Folge,Philipp Wilhelmer,564,299802565
1256912942211654061,299982579,News,VIDEO: Basejumper springen von Berg in Flugzeug,Mathias Kainz,574,299965853
1266050781470528842,299934123,News,Serienräuber schlugen mindestens zwölf Mal in Wien und NÖ zu,,574,299941050
1711062148435509764,201704202,Stars & Kultur,"Naddel Ab ed Farrag: ""Lebe von 400 Euro im Monat""",Elisabeth Spitzer,556,181777048
1741416945750781669,299446027,Lifestyle,Pädophilie: Youtube-Werbung wegen Kinderbildern gestoppt,Elisabeth Mittendorfer,574,299826775
175237463